 When you are done installing CIAO import the following package to get started. 
 
Also it is recommended to install astropy in the same conda environment because sometimes astropy will help you to check the header of the given fits files

In [ ]:
import ciao_contrib.runtool as rt
from astropy.io import fits

In [ ]:
#Load the event file onto which bary center correction is to be applied
event_filename = '<event file name>'

#Use astropy to to get the RA and DEC 
#You can also use something else to view the header and get RA and DEC
events_list = fits.open(event_filename)[1]
ra = events_list.header['RA_PNT']
dec = events_list.header['DEC_PNT']

### Barycenter correction on the evt2 file

In [ ]:
# For bary center correction we need to use axbary task
# Setting up the parameters of axbary
rt.axbary.punlearn()
rt.axbary.infile = event_filename
rt.axbary.orbitfile = '<orbit file name>'
outfile_name = f'bary_{event_filename}' # this naming convention I use. you can put name of your choice 
rt.axbary.outfile = outfile_name 
rt.axbary.ra = ra #got from the header
rt.axbary.dec = dec #got from the header
rt.axbary.clobber = 'yes' #by default it is 'no'; 'yes' means if a file with the outfile name already exists it will overwrite

#Checking the parameters after all set
print(rt.axbary)

In [ ]:
#Running the task
rt.axbary()

### Doing same thing with asolfile (only infile, outfile will be changed)

Bary center correction is already done to the evt2 file. But sometimes it is needed to have the aspect solution file (for example while extracting speectrum). In those cases aspect solution file also should be applied the correction. 

In [ ]:
#changing some of the parameters of axbary
input_asol_file = input('Enter the input asol file')
rt.axbary.infile = input_asol_file
rt.axbary.outfile = f'bary_{input_asol_file}'  # this naming convention I use. you can put name of your choice 

# Checking the parameters after necessary changes (notice rest of the parameters are kept as it is)
print(rt.axbary)    

In [ ]:
#running the task
rt.axbary()

Now that we have both bary centere corrected event file and aspect solution file, in the header of event file I need to mention which aspect solution file to be used when it is needed (for example while spectrum extraction). By default the 'not corrected' aspect solution file would be there, which I need to replace by 'corrected' aspect solution file. 

The key word in the header is "ASOLFILE" 

The task we need is 'dmhedit'

### Changing the header keyword of the Bary center corrected evt2 file

In [ ]:
#Setting the parameters of the dmhedit task
rt.dmhedit.infile = outfile_name #The variable which contains the bary center corrected event file (outfile of axbary for the first task)
rt.dmhedit.operation = 'add'
rt.dmhedit.key = 'ASOLFILE'
rt.dmhedit.value = rt.axbary.outfile #The outfile of the axbary for the second (last) task, which is the corrected aspect solution file

#Checking the parameters of the task after all set
print(rt.dmhedit)

In [ ]:
# Running the task
rt.dmhedit()


                                             Now you are done with bary center correction